In [11]:
import nltk
from pprint import pprint as print
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath
from gensim.models.word2vec import Word2Vec, Text8Corpus
import numpy as np
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Prime prove fastText
il modello creato è stato creato con 1 epoca dal modello wikipedia, da migliorare e diminuire le parole, 
introducendo un dizionario di paroli comuni italiane (vedi come fa il professore)

le funzioni seguenti servono per fare lo spell checker e non per predirre la parola sucessiva

In [13]:
model = Word2Vec.load('WikiW2C1.model') #modello creato con fast text 1 epoca (da migliorare)

2020-04-02 19:31:58,763 : INFO : loading Word2Vec object from WikiW2C1.model
2020-04-02 19:31:59,276 : INFO : loading wv recursively from WikiW2C1.model.wv.* with mmap=None
2020-04-02 19:31:59,277 : INFO : setting ignored attribute vectors_norm to None
2020-04-02 19:31:59,277 : INFO : loading vocabulary recursively from WikiW2C1.model.vocabulary.* with mmap=None
2020-04-02 19:31:59,278 : INFO : loading trainables recursively from WikiW2C1.model.trainables.* with mmap=None
2020-04-02 19:31:59,278 : INFO : setting ignored attribute cum_table to None
2020-04-02 19:31:59,279 : INFO : loaded WikiW2C1.model


In [14]:
path = '/Users/stefanoraimondousai/Documents/ReadingCourse/text_pulito.txt' #testo con chat marta
sentences = datapath(path)

In [15]:
model.train(sentences,epochs=5,total_examples=model.corpus_count, compute_loss=False)

2020-04-02 19:32:03,572 : INFO : training model with 8 workers on 63937 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2020-04-02 19:32:03,579 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-04-02 19:32:03,579 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-04-02 19:32:03,579 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-04-02 19:32:03,580 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-04-02 19:32:03,580 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-02 19:32:03,581 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-02 19:32:03,581 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-02 19:32:03,582 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-02 19:32:03,583 : INFO : EPOCH - 1 : training on 66 raw words (45 effective words) took 0.0s, 7121 effective words

(230, 330)

In [16]:
dictionary = list(model.wv.vocab.keys())

In [17]:
    def predict_output_word(model, context_words_list, topn=10):
        from numpy import exp, dot, zeros, random, dtype, float32 as REAL,\
        uint32, seterr, array, uint8, vstack, fromstring, sqrt,\
        empty, sum as np_sum, ones, logaddexp, log, outer
        from scipy.special import expit
        from gensim import utils, matutils  # utility fnc for pickling, common scipy operations etc

        if not model.negative:
            raise RuntimeError(
                "We have currently only implemented predict_output_word for the negative sampling scheme, "
                "so you need to have run word2vec with negative > 0 for this to work."
            )

        if not hasattr(model.wv, 'vectors') or not hasattr(model.trainables, 'syn1neg'):
            raise RuntimeError("Parameters required for predicting the output words not found.")

        word_vocabs = [model.wv.vocab[w] for w in context_words_list if w in model.wv.vocab]
        if not word_vocabs:
            warnings.warn("All the input context words are out-of-vocabulary for the current model.")
            return None

        word2_indices = [word.index for word in word_vocabs]

        l1 = np_sum(model.wv.vectors[word2_indices], axis=0)
        if word2_indices and model.cbow_mean:
            l1 /= len(word2_indices)

        # propagate hidden -> output and take softmax to get probabilities
        prob_values = exp(dot(l1, model.trainables.syn1neg.T))
        prob_values /= sum(prob_values)
        top_indices = matutils.argsort(prob_values, topn=topn, reverse=True)
        # returning the most probable output words with their probabilities
        return [(model.wv.index2word[index1], prob_values[index1]) for index1 in top_indices]

In [18]:
def weight(word, similar_word, similarity, constant = 0.5, min_weight=3):
    min_similarity = similarity > constant   
    word_len = 1 if len(word) <= 4 else 2 #se è minore di 4 è un articolo quasi sicuramente
    exist = 1 if word in dictionary else 0 # esiste nel dizionario?
    min_len = len(similar_word) > 4 #se è più lungo di 4 evito che siano articoli
    check_first_word = 1 if word[0] == similar_word[0] else 0 #controllo la prima lettera della parola
    levenshtein_distance = nltk.edit_distance(word, similar_word) <= word_len #sostituire con quello di marta
    weight = word_len + min_similarity + min_len + exist + levenshtein_distance
    if weight > min_weight:
        return True
    else:
        return False
    

In [19]:
def spellChecker(word):
    most_similar = model.wv.most_similar(word, topn=50) #parole simili dal modello
    result = list(filter(lambda x: weight(word, x[0], x[1]), most_similar))#filtro con la funzione weight
    result = {'prediction': result[:3]} #dizionario con le prime 5 parole
    return result

In [20]:
spellChecker('prpfessore')

2020-04-02 19:32:12,523 : INFO : precomputing L2-norms of word weight vectors


KeyError: "word 'prpfessore' not in vocabulary"

In [22]:
predict_output_word(model, ['giovedì', 'andiamo', 'al'], topn=50)

[('maresciallo', 0.0025712752),
 ('maciel', 0.0022833087),
 ('generale', 0.0020807434),
 ('marco', 0.0019856805),
 ('ore', 0.0019567062),
 ('vangelo', 0.0018323088),
 ('napoleone', 0.0015985918),
 ('ottobre', 0.001584941),
 ('00', 0.0014943439),
 ('gesù', 0.0014279209),
 ('mattino', 0.0011107996),
 ('alle', 0.00086087454),
 ('soldati', 0.0008585797),
 ('ordine', 0.0008323802),
 ('verso', 0.0007936085),
 ('profeta', 0.00075703964),
 ('14', 0.00074982725),
 ('pomeriggio', 0.00057773816),
 ('truppe', 0.0005752581),
 ('versetto', 0.00056621875),
 ('fianco', 0.000548455),
 ('momento', 0.0005382026),
 ('moro', 0.00050576596),
 ('mosè', 0.0005003582),
 ('armata', 0.00049785496),
 ('principe', 0.0004932077),
 ('lunedì', 0.0004915489),
 ('102', 0.00046582235),
 ('000', 0.00045566593),
 ('capodanno', 0.00045552698),
 ('mentre', 0.0004422876),
 ('voi', 0.00044033604),
 ('voto', 0.00042722374),
 ('corpo', 0.00042338882),
 ('radio', 0.00041576417),
 ('battaglia', 0.0004143782),
 ('13', 0.0004021185